<h1 style="border: 5px dotted red; padding: 10px;font-size: 75px;text-align:center">Welcome to the presentation of the next generation of ShopperzЯUs shopping cart recommendation program!!!</h1>

<h2 style="text-align:center">Below you can see a few graphs demonstrating the percentages of customers who purchased a particular item. </h2>
<p style="color:orange;font-size: 150px;text-align:center;border: 5px solid yellow"> &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; </p>

In [147]:
# insert graph of top 10 purchased items here... TO DO
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import csv
import matplotlib as plt
from PyScripts.InventoryBook import InventoryBook


product_rules = pd.read_csv('Data/Master_Rules.csv')

In [148]:
with open('Data/Master_Names.csv',encoding='utf-8') as f:
    read = csv.reader(f)
    next(read)
    names = list(read)

In [149]:
names_updated = list(map(lambda x: x[1],names))

In [150]:
names_updated.sort()
master_inventory = None

In [166]:
'''
Master list of widgets below:
'''
style = {'description_width': 'initial', 'width' : 'max-content'}
inventory_amount_choice = widgets.Select(options=[x for x in range(1,11)],value=1, description='INV amount: ',disabled=False)
inventory_need_qualifier = widgets.RadioButtons(options=['DAILY','WEEKLY','MONTHLY','RARELY'],desciption='Projected Usage of Item',disabled=False)
inventory_list_choices = widgets.Select(options=names_updated,value=names_updated[0],description='Choose Item To Add: ',style=style,disabled=False)
inventory_random_add = widgets.Button(description="Select Random Item",disabled=False, button_style='info',tooltip='Click me to add a random item to the inventory',icon='random')
inventory_save_button = widgets.Button(descriptio='Save To Inventory',disabled=True, button_style='success',tooltip='Click me to save chosen item with amount to inventory',icon='save')
inventory_output = widgets.Output(layout={'border':'5px solid black'}) #TODO add default value
recommendation_output = widgets.Output(value='***Recommendations Go Here***' ,layout={'border':'5px solid black'})
add_user = widgets.Button(description="Create Inventory",disabled=False, button_style='success',tootip='Click me to create a new Inventory Book with the provided owners name.',icon='user-plus')
user_name_input = widgets.Text(value="What's in a name?",description='Enter User Name: ',style=style,disabled=False)

# left_vbox = widgets.VBox(children=[inventory_random_add,inventory_list_choices])
# middle_hbox = widgets.HBox(children=[inventory_amount_choice])
# right_hbox = widgets.HBox(children=[inventory_need_qualifier])
# wrapper_vbox = widgets.VBox(children=[[left_vbox, middle_hbox, right_hbox],[ inventory_save_button]])
# user_input_hbox = widgets.HBox(children=[user_name_input, add_user])
#TODO ... add .on_click method for buttons to process data correctly. capture the button click event

In [167]:
def create_inventory(x):
    global master_inventory
    master_inventory = InventoryBook(user_name_input.value)
    user_name_input.close()
    add_user.close()
    inventory_save_button.disabled = False

def update_inventory(y):
    global master_inventory
    values = [str(inventory_list_choices.value),str(inventory_need_qualifier.value) , int(inventory_amount_choice.value), ]
    master_inventory.add_item(values)

@recommendation_output.capture(wait=True)
def create_recommendation(z):
    global master_inventory
    with recommendation_output:
        text = master_inventory.create_recommendation()
        print(text)

In [168]:
add_user.on_click(create_inventory)
inventory_save_button.on_click(update_inventory)

In [169]:
inventory_list_choices.observe(create_recommendation)

In [170]:
display(user_name_input)
display(add_user)

Text(value="What's in a name?", description='Enter User Name: ', style=DescriptionStyle(description_width='ini…

Button(button_style='success', description='Create Inventory', icon='user-plus', style=ButtonStyle())

In [171]:
display(inventory_list_choices)

Select(description='Choose Item To Add: ', options=('#2 Coffee Filters', '#2 Cone White Coffee Filters', '#2 M…

In [172]:
display(inventory_need_qualifier)

RadioButtons(options=('DAILY', 'WEEKLY', 'MONTHLY', 'RARELY'), value='DAILY')

In [173]:
display(inventory_amount_choice)

Select(description='INV amount: ', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=1)

In [174]:
display(inventory_random_add)
display(inventory_save_button)

Button(button_style='info', description='Select Random Item', icon='random', style=ButtonStyle(), tooltip='Cli…

Button(button_style='success', disabled=True, icon='save', style=ButtonStyle(), tooltip='Click me to save chos…

In [175]:
display(inventory_output)

Output(layout=Layout(border='5px solid black'))

In [176]:
display(recommendation_output)

Output(layout=Layout(border='5px solid black'))

<p>By utilizing the previous shoppers habits as training data, our new ShoppingKart app will display <b>real-time</b> recommendations for additional items to add to the user's inventory.</p>

<p> This demonstration will show the recommendation portion of the app off. The development team at ShopperzЯUs are finishing up the Android based app for our customers to utilize daily for their grocery inventory, and shopping needs!! </p>

<p> The item database is over <strong>30,000</strong> unique items that can be added. The android app is going to process the input from the customer when creating their inventory to match up to the best item we have availble in order to make re-ordering and also upselling a breeze for the customer and profits for the company!! </p>

<p> Below is a mock inventory builder. You can either select an item from the list, or add a random item from our inventory base to the inventory list along with the amount currently on hand. This system will dynamically generate both, the top recommended item(s) based on what has been entered, and will also generate the data associated behind the machine-learning based process of assosication that creates the recommondations via graphs. It also displays the confidence level for the recommendation that was made.</p>

In [162]:
# put GUI here TO DO


<p style="border: 5px solid red; font-size: 35px; text-align:center; padding: 15px"> We are anxiously awaiting the completion of the Android app to fully utilize the recommendation program and ensure better profits and more sales and happy customers!!!</p>

<div style="float: left; text-align:center; border: 5px dotted purple; padding: 15px; font-size: 30px">CopyRight &copy; Catch22Software 2022</div>
<div style="text-align:center; padding: 15px">
<img src="Images/curveball.jpg" alt="Company Logo" width="200" height="200"></div>